In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from copy import copy

if not os.path.exists("details.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/13_CCL/details.png --no-check-certificate
details = cv2.imread('details.png', cv2.IMREAD_GRAYSCALE)

plt.imshow(details, 'gray')
plt.axis('off')
plt.title('Obraz oryginalny')
plt.show()

In [ ]:
fig, ax = plt.subplots(2,4,figsize=(30,10))

dilate = cv2.dilate(details, cv2.getStructuringElement(cv2.MORPH_RECT, (7,7)))
background = cv2.medianBlur(dilate, 17)
diff = 255 - cv2.absdiff(details, background)
norm = cv2.normalize(diff, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)

ax[0][0].imshow(norm, 'gray')
ax[0][0].axis('off')
ax[0][0].set_title('1. Usunięcie tła z cieniami')

details_tophat_1 = cv2.morphologyEx(norm, cv2.MORPH_TOPHAT, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15)))
details_tophat_segment_1 = np.zeros(details_tophat_1.shape)
details_tophat_segment_1[50 : 130, 60 : 130] = details_tophat_1[50 : 130, 60 : 130]
details_tophat_segment_1 = cv2.normalize(details_tophat_segment_1, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
ax[0][1].imshow(details_tophat_segment_1, 'gray')
ax[0][1].axis('off')
ax[0][1].set_title('2. Wyeksponowanie pierwszego białego elementu na czarnym tle')

details_tophat_2 = cv2.morphologyEx(norm, cv2.MORPH_TOPHAT, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8)))
details_tophat_segment_2 = np.zeros(details_tophat_2.shape)
details_tophat_segment_2[130 : 160, 60 : 130] = details_tophat_2[130 : 160, 60 : 130]
details_tophat_segment_2 = cv2.normalize(details_tophat_segment_2, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
ax[0][2].imshow(details_tophat_segment_2, 'gray')
ax[0][2].axis('off')
ax[0][2].set_title('3. Wyeksponowanie drugiego białego elementu na czarnym tle')

details_blackhat = cv2.morphologyEx(norm, cv2.MORPH_BLACKHAT, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15)))
details_blackhat_segment = details_blackhat
details_blackhat[50 : 160, 60 : 130] = 0
ax[0][3].imshow(details_blackhat_segment, 'gray')
ax[0][3].axis('off')
ax[0][3].set_title('4. Wyeksponowanie pozostałych czarnych elementów na białym tle')

sum = details_tophat_segment_1 + details_tophat_segment_2 + details_blackhat_segment
ax[1][0].imshow(sum, 'gray')
ax[1][0].axis('off')
ax[1][0].set_title('5. Zsumowanie powyższych obrazów')

bin = sum > 55
ax[1][1].imshow(bin, 'gray')
ax[1][1].axis('off')
ax[1][1].set_title('6. Binaryzacja')

close = cv2.morphologyEx(bin.astype('uint8'), cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4)), iterations=1)
ax[1][2].imshow(close, 'gray')
ax[1][2].axis('off')
ax[1][2].set_title('7. Zamknięcie')

erode = cv2.erode(close, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2)))
ax[1][3].imshow(erode, 'gray')
ax[1][3].axis('off')
ax[1][3].set_title('8. Erozja')

plt.show()

In [ ]:
num_labels, new_erode, stats, centroids = cv2.connectedComponentsWithStats(erode)
plt.imshow(new_erode)
plt.axis('off')
plt.title('Rozpoznawanie')
plt.show()

vis_details = new_erode.copy()
objects = []
moments = []
HuMoments = []
for i in range(1, num_labels):
    object = (new_erode == i) * 1
    object = object.astype('uint8')
    objects.append(object)
    moment = cv2.moments(object)
    # print(i)
    # print(moment['m00'])
    moments.append(moment)
    HuMoment = cv2.HuMoments(moment)
    # print(HuMoment[0])
    # print()
    HuMoments.append(HuMoment)
    cv2.putText(vis_details,str(i),(int(centroids[i,0]),int(centroids[i,1])),cv2.FONT_HERSHEY_SIMPLEX,1,(20,0,0),2)
plt.imshow(vis_details)
plt.axis('off')
plt.title('Numeracja')
plt.show()

In [ ]:
new_image = np.zeros(new_erode.shape)
for i in range(1, num_labels):
    if HuMoments[i - 1][0] > 0.27 and moments[i - 1]['m00'] > 413:
        new_image += objects[i - 1]

plt.imshow(new_image)
plt.axis('off')
plt.title('Śrubki')
plt.show()

new_image = np.zeros(new_erode.shape)
for i in range(1, num_labels):
    if (HuMoments[i - 1][0] < 0.23 and moments[i - 1]['m00'] < 250) or (HuMoments[i - 1][0] < 0.205 and 727 > moments[i - 1]['m00'] > 634):
        new_image += objects[i - 1]

plt.imshow(new_image)
plt.axis('off')
plt.title('Nakrętki')
plt.show()

new_image = np.zeros(new_erode.shape)
for i in range(1, num_labels):
    if not(HuMoments[i - 1][0] > 0.27 and moments[i - 1]['m00'] > 413) and not ((HuMoments[i - 1][0] < 0.23 and moments[i - 1]['m00'] < 250) or (HuMoments[i - 1][0] < 0.205 and 727 > moments[i - 1]['m00'] > 634)):
        new_image += objects[i - 1]

plt.imshow(new_image)
plt.axis('off')
plt.title('Podkładki')
plt.show()